# 🚀 Autonomous Trading System Training Notebook

This notebook provides an interactive interface for training and analyzing autonomous trading agents with advanced visualization capabilities.

## Features:
- 🧠 **Autonomous Agent Training**: Self-evolving neural architectures with hyperparameter tuning
- 📊 **Real-time Visualization**: Training progress, fitness evolution, and performance metrics
- 🔄 **Complete Training Pipeline**: PPO → MoE → MAML → Autonomous
- 💰 **Risk Management**: Integrated 1:2 risk-reward ratio with ATR-based stops
- 🎯 **Champion Selection**: Automatic saving of best-performing agents

---

## 📦 Setup and Imports

In [1]:
#!/usr/bin/env python3
"""
Autonomous Trading System Training Notebook
Interactive training and analysis for self-evolving trading agents
"""

import os
import sys
import logging
import warnings
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import trading system components
from src.training.parallel_trainer import ParallelTrainer
from src.training.parallel_config import ParallelTrainingConfig, get_recommended_config
from src.training.trainer import Trainer
from src.training.sequence_manager import TrainingSequenceManager
from src.training.autonomous_trainer import AutonomousTrainer, AutonomousTrainingConfig, run_autonomous_stage
from src.agents.ppo_agent import PPOAgent
from src.agents.moe_agent import MoEAgent
from src.agents.autonomous_agent import AutonomousAgent
from src.backtesting.environment import TradingEnv
from src.utils.data_loader import DataLoader
from src.config.config import INITIAL_CAPITAL

# Configure logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Autonomous Trading System Initialized!")
print(f"💰 Initial Capital: ₹{INITIAL_CAPITAL:,.2f}")
print(f"📅 Session Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

NameError: name 'MultiHeadTransformerModel' is not defined

## 🔧 Configuration and Data Discovery

In [ ]:
def get_available_symbols(data_dir: str = "data/final") -> List[str]:
    """Get list of available trading symbols from data directory."""
    symbols = []
    
    if not os.path.exists(data_dir):
        logger.warning(f"Data directory {data_dir} does not exist")
        return symbols

    for filename in os.listdir(data_dir):
        if filename.endswith('.csv') and filename.startswith('features_'):
            symbol = filename.replace('features_', '').replace('.csv', '')
            symbols.append(symbol)
    
    return sorted(symbols)

# Discover available data
DATA_DIR = "data/final"
available_symbols = get_available_symbols(DATA_DIR)

print(f"📊 Found {len(available_symbols)} available symbols:")
for i, symbol in enumerate(available_symbols, 1):
    print(f"  {i:2d}. {symbol}")

# Create interactive symbol selector
symbol_selector = widgets.SelectMultiple(
    options=available_symbols,
    value=[available_symbols[0]] if available_symbols else [],
    description='Symbols:',
    disabled=False,
    style={'description_width': 'initial'}
)

algorithm_selector = widgets.Dropdown(
    options=['Autonomous', 'PPO', 'MoE', 'MAML'],
    value='Autonomous',
    description='Algorithm:',
    style={'description_width': 'initial'}
)

training_mode = widgets.Dropdown(
    options=['sequence', 'simple', 'parallel'],
    value='sequence',
    description='Mode:',
    style={'description_width': 'initial'}
)

episodes_slider = widgets.IntSlider(
    value=50,
    min=1,
    max=1000,
    step=10,
    description='Episodes/Generations:',
    style={'description_width': 'initial'}
)

display(widgets.VBox([
    widgets.HTML("<h3>🎛️ Training Configuration</h3>"),
    symbol_selector,
    algorithm_selector,
    training_mode,
    episodes_slider
]))

📊 Found 3 available symbols:
   1. Bank_Nifty_5
   2. Nifty_2
   3. Sensex_15


## 📈 Data Analysis and Visualization

In [ ]:
def analyze_symbol_data(symbol: str, data_dir: str = "data/final") -> Dict[str, Any]:
    """Analyze and visualize symbol data."""
    try:
        # Load data
        data_path = os.path.join(data_dir, f"features_{symbol}.csv")
        df = pd.read_csv(data_path)
        
        # Basic statistics
        stats = {
            'symbol': symbol,
            'total_rows': len(df),
            'date_range': (df['timestamp'].min(), df['timestamp'].max()) if 'timestamp' in df.columns else ('N/A', 'N/A'),
            'features': len(df.columns),
            'missing_values': df.isnull().sum().sum(),
            'price_columns': [col for col in df.columns if any(x in col.lower() for x in ['price', 'close', 'open', 'high', 'low'])]
        }
        
        return stats, df
    except Exception as e:
        logger.error(f"Error analyzing {symbol}: {e}")
        return None, None

def plot_symbol_overview(symbol: str, df: pd.DataFrame):
    """Create comprehensive visualization of symbol data."""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'{symbol} Price Action',
            'Feature Distribution',
            'Volume Analysis',
            'Volatility Patterns'
        ),
        specs=[[{"secondary_y": True}, {}],
               [{}, {}]]
    )
    
    # Price action (if available)
    price_cols = [col for col in df.columns if any(x in col.lower() for x in ['close', 'price'])]
    if price_cols:
        price_col = price_cols[0]
        fig.add_trace(
            go.Scatter(x=df.index, y=df[price_col], name='Price', line=dict(color='blue')),
            row=1, col=1
        )
    
    # Feature distribution
    numeric_cols = df.select_dtypes(include=[np.number]).columns[:10]  # Top 10 numeric features
    for col in numeric_cols:
        fig.add_trace(
            go.Histogram(x=df[col], name=col, opacity=0.7),
            row=1, col=2
        )
    
    # Volume analysis (if available)
    volume_cols = [col for col in df.columns if 'volume' in col.lower()]
    if volume_cols:
        volume_col = volume_cols[0]
        fig.add_trace(
            go.Bar(x=df.index[-100:], y=df[volume_col].iloc[-100:], name='Volume'),
            row=2, col=1
        )
    
    # Volatility patterns
    if price_cols:
        returns = df[price_cols[0]].pct_change().dropna()
        rolling_vol = returns.rolling(window=20).std() * np.sqrt(252)  # Annualized volatility
        fig.add_trace(
            go.Scatter(x=rolling_vol.index, y=rolling_vol, name='Rolling Volatility'),
            row=2, col=2
        )
    
    fig.update_layout(
        height=800,
        title_text=f"📊 {symbol} Data Analysis Overview",
        showlegend=True
    )
    
    return fig

# Analyze selected symbol
if available_symbols:
    selected_symbol = available_symbols[0]  # Default to first symbol
    stats, df = analyze_symbol_data(selected_symbol)
    
    if stats and df is not None:
        print(f"\n📊 Analysis for {selected_symbol}:")
        print(f"  📈 Total Data Points: {stats['total_rows']:,}")
        print(f"  🔢 Features: {stats['features']}")
        print(f"  📅 Date Range: {stats['date_range'][0]} to {stats['date_range'][1]}")
        print(f"  ❌ Missing Values: {stats['missing_values']}")
        
        # Create and display visualization
        fig = plot_symbol_overview(selected_symbol, df)
        fig.show()
    else:
        print(f"❌ Could not analyze {selected_symbol}")
else:
    print("❌ No symbols available for analysis")